In [279]:
import requests
from lxml import etree
import pandas as pd

In [280]:
url = 'https://movie.douban.com/subject/35613853/comments?status=P'
headers ={ 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.81 Safari/537.36'}
res = requests.get(url, headers=headers)
res

<Response [200]>

In [281]:
txt = res.text

In [282]:
dom = etree.HTML(txt)
names = dom.xpath('//span[@class="comment-info"]/a/text()')
ratings = dom.xpath('//span[@class="comment-info"]/span[starts-with(@class,"allstar")]/@class')
#或者
#ratings = dom.xpath('//span[@class="comment-info"]/span[2]/@class')

#class值中有空格。
times = dom.xpath('//span[@class="comment-info"]/span[@class="comment-time "]/@title')
#这里第二个//代表子孙节点即可
message = dom.xpath('//div[@class="comment"]//span[@class="short"]/text()')
user_url= dom.xpath('//span[@class="comment-info"]/a/@href') #用户名的超链接
votes = dom.xpath('//div[@class="comment"]//span[@class="votes vote-count"]/text()')

In [283]:
import random
cities = []
load_times = []
j = 0
for i in user_url:
    print(f'正在爬取第{j}页')
    j +=1 
    web_data = requests.get(i, headers=headers)
    dom_url = etree.HTML(web_data.text, etree.HTMLParser(encoding='utf-8'))
    #为了保险起见，可以把div的上级div也写上
    address = dom_url.xpath('//div[@class="basic-info"]//div[@class="user-info"]/a/text()')
    load_time = dom_url.xpath('//div[@class="basic-info"]//div[@class="user-info"]/div[@class="pl"]/text()')
    cities.append(address)
    load_times.append(load_time)
    time.sleep(random.randint(2,4))

正在爬取第0页
正在爬取第1页
正在爬取第2页
正在爬取第3页
正在爬取第4页
正在爬取第5页
正在爬取第6页
正在爬取第7页
正在爬取第8页
正在爬取第9页
正在爬取第10页
正在爬取第11页
正在爬取第12页
正在爬取第13页
正在爬取第14页
正在爬取第15页
正在爬取第16页
正在爬取第17页
正在爬取第18页
正在爬取第19页


In [284]:
print(names)
print(ratings)
print(times)
print(message)
print(user_url)
print(votes)
print(cities)
print(load_times)

['余霞鸿雁', '雾散', '李六岁的娃娃亲', 'ERic', '缅怀', '王祉乔', '单数路人甲', 'forcedemail.', '大象', 'Anna', '𝔹𝕒𝕒 𝔹𝕒𝕒 𝕊𝕠𝕟𝕘', '六尺之上', 'Papercut', '张3儿', '滕暴富', '勇气', '阿鑫.', '霖', 'ZYX1607', '拿卡利']
['allstar20 rating', 'allstar50 rating', 'allstar50 rating', 'allstar20 rating', 'allstar30 rating', 'allstar40 rating', 'allstar40 rating', 'allstar50 rating', 'allstar10 rating', 'allstar30 rating', 'allstar30 rating', 'allstar30 rating', 'allstar20 rating', 'allstar10 rating', 'allstar50 rating', 'allstar30 rating', 'allstar40 rating', 'allstar40 rating', 'allstar40 rating', 'allstar50 rating']
['2022-02-02 14:47:12', '2022-02-01 10:56:25', '2022-02-01 11:14:14', '2022-02-02 10:56:00', '2022-02-01 11:25:55', '2022-02-01 11:19:11', '2022-02-01 14:10:10', '2022-02-01 10:57:35', '2022-02-02 18:11:05', '2022-02-01 10:49:22', '2022-02-01 11:43:00', '2022-02-01 22:53:11', '2022-02-02 21:29:22', '2022-02-06 17:00:15', '2022-02-01 11:15:40', '2022-02-02 21:35:57', '2022-02-01 10:43:06', '2022-02-01 10:58:50', '2022-02-01 

In [285]:
print(len(names))
print(len(ratings))
print(len(times))
print(len(message))
print(len(user_url))
print(len(votes))
print(len(cities))
print(len(load_times))

20
20
20
20
20
20
20
20


In [286]:
import re
ratings_n = ['' if 'rating' not in i else int(re.findall(r'[0-9]+', i)[0]) for i in ratings] #也可以re.findall('\d{2}', i)
cities_n = ['' if i == [] else i[0] for i in cities]
load_times_n = ['' if i == [] else i[1][:-2].strip(' ') for i in load_times]

In [287]:
#构建数据框
data = pd.DataFrame({
    '用户名': names,
    '居住城市': cities,
    '加入时间': load_times,
    '评分': ratings,
    '发表时间': times,
    '短评正文': message,
    '赞同数量': votes
})
data

,用户名,居住城市,加入时间,评分,发表时间,短评正文,赞同数量
0,余霞鸿雁,[],[],allstar20 rating,2022-02-02 14:47:12,5星给抗美援朝题材。1星扣电影里的失智桥段就是硬煽情硬壮烈，1星扣不合逻辑，美军支援直接3个...,1536
1,雾散,[],[],allstar50 rating,2022-02-01 10:56:25,“第七穿插连应到一百五十七人，实到一人。”,6333
2,李六岁的娃娃亲,[北京],"[64774468 , 2012-09-19加入]",allstar50 rating,2022-02-01 11:14:14,七连全军就义只剩一面旗，万里能留下来全因为是大年初一。,3158
3,ERic,[],[],allstar20 rating,2022-02-02 10:56:00,不煽情会更好,1161
4,缅怀,[],[],allstar30 rating,2022-02-01 11:25:55,吴京终于以主角光环➕360度慢镜头全方位无死角的方式展示了他的牺牲,6161
5,王祉乔,[上海],"[156344445 , 2017-01-13加入]",allstar40 rating,2022-02-01 11:19:11,1950年 是新中国成立后的第一个虎年,5705
6,单数路人甲,[],[],allstar40 rating,2022-02-01 14:10:10,一代人打了五代人仗，受了五代人的苦,3999
7,forcedemail.,[],[],allstar50 rating,2022-02-01 10:57:35,易烊千玺饰演的万里的成长线真的很让人心疼，战争让一个少年迅速衰老三十岁…,2829
8,大象,[],[],allstar10 rating,2022-02-02 18:11:05,如果已经不用再歌唱那远去的少年，\n杯光碟影的乐土，\n我只能告诉你，\n最夺目并最操蛋的都...,512
9,Anna,[],[],allstar30 rating,2022-02-01 10:49:22,在这个时代的主旋律下，每一个观众或多或少能找到些共鸣。但是缺少创新的思想，一贯老套的视听，没...,2662
